<a href="https://colab.research.google.com/github/ra4ola/APA-Assigment2/blob/main/Parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from torch.nn.modules.flatten import Flatten
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# Defenição de Variáveis

In [ ]:
PATH = '/content/drive/MyDrive/APA/Assigment2/Parte3'

In [ ]:
image_size = 28
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NameError: ignored

# Definição de Funções

### Outras

In [ ]:
def ModelDirStr(modelName,path = PATH):
  return f"{path}/{modelName}"
def pathModelStr(modelName,epoch, path = PATH):
  return f"{ModelDirStr(modelName,path)}/{modelName}_epoch={epoch}"

## *AE*




### Função de Treino *AE*


In [ ]:
def AEtrain(model,criterion,optimizer,train_loader,epochs,modelName,scheduler=None, path = PATH):
  os.makedirs(ModelDirStr(modelName= modelName,path= path), exist_ok=True)
  for epoch in range(epochs):
    for data in train_loader:
        inputs, _ = data
        inputs = inputs.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

    if scheduler is not None:
      scheduler.step()

    torch.save(model, pathModelStr(modelName= modelName,path = path, epoch = epoch))



## *VAE*

### Função para adicionar Noise

In [ ]:
def add_noise(data, noise_factor=0.5):
    noisy_data = data + noise_factor * torch.randn_like(data)
    return torch.clamp(noisy_data, 0., 1.)


### Função de Treino *VAE*

#### Kullback-Leibler Divergence (KLD) Loss:

$$ D_{KL} = -\frac{1}{2} \sum_{i} (1 - \mu_{i}^{2} + \log(\sigma_{i}^{2}) - e^{\log(\sigma_{i}^{2})}) $$


In [ ]:
def VAEtrain(model,criterion,optimizer,train_loader,epochs,modelName,scheduler=None, path = PATH):
  os.makedirs(ModelDirStr(modelName= modelName,path= path), exist_ok=True)
  for epoch in range(epochs):
    for data in train_loader:
      inputs, _ = data
      inputs = inputs.to(device)
      optimizer.zero_grad()
      outputs, mean, log_var = model(add_noise(inputs))

      reconstruction_loss = criterion(outputs, inputs)
      kld_loss = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
      loss = reconstruction_loss + kld_loss

      loss.backward()
      optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

    if scheduler is not None:
      scheduler.step()

    torch.save(model, pathModelStr(modelName= modelName,path = path, epoch = epoch))

## Função de Implementação das Perdas


In [ ]:
def evaluate_training(training_loader, loss_fn, num_epochs,modelName, path = PATH):
    all_losses = []
    all_accuracies = []

    for epoch in range(num_epochs):
        # Load the model
        model_path = pathModelStr(modelName = modelName, path = path)
        model = torch.load(model_path)
        model.to(device)
        model.eval()

        # Evaluate the model
        valid_loss = 0.0
        num_correct = 0
        num_examples = 0

        with torch.no_grad():
            for batch in training_loader:
                inputs, targets = batch
                inputs = inputs.to(device)
                targets = targets.to(device)
                output = model(inputs)
                loss = loss_fn(output, targets)
                valid_loss += loss.item()

                preds = torch.argmax(output, dim=1)
                correct = preds.eq(targets)
                num_correct += correct.sum().item()
                num_examples += correct.shape[0]

        valid_loss /= len(training_loader)

        accuracy = num_correct / num_examples
        all_losses.append(valid_loss)
        all_accuracies.append(accuracy)

        print(f'Epoch {epoch}: Validation Loss: {valid_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')

    return all_losses, all_accuracies


In [ ]:
import matplotlib.pyplot as plt

def plot_loss_accuracy(losses, accuracies):
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, 'b', label='Validation Loss')
    plt.title('Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, 'r', label='Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def reconstruction_images_plot(model,test_loader):
  with torch.no_grad():
    for data in test_loader:
      inputs, _ = data
      inputs = inputs.to(device)

      outputs = model(inputs)
      # Display original and reconstructed images
      n = 10  # Number of digits to display
      plt.figure(figsize=(20, 4))
      for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(inputs[i].cpu().view(28, 28).numpy(), cmap='gray')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(outputs[i].cpu().view(28, 28).numpy(), cmap='gray')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

      plt.show()
      break


# Importação do DataSet

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 153846223.92it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 123987404.12it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 54992775.10it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12863287.49it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Modelos


## Auto-Encoder Convolution Layer

In [ ]:
class AEconv(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder_conv = nn.Sequential(

            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(8),
            nn.ELU(),

            nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ELU(),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ELU(),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ELU(),

            nn.Conv2d(in_channels=128, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ELU()
        )
        self.decoder_conv = nn.Sequential(

         nn.ConvTranspose2d(in_channels=16, out_channels=128, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(128),
         nn.ELU(),

         nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(64),
         nn.ELU(),

         nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(32),
         nn.ELU(),

         nn.ConvTranspose2d(in_channels=32, out_channels=8, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(8),
         nn.ELU(),

         nn.Upsample(scale_factor=2, mode='nearest'),  # Upsample
         nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(1),
         nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder_conv(x)
        x = self.decoder_conv(x)
        return x

## VAE Encoder Convolution


In [ ]:
class nnModel(nn.Module):
    def __init__(self):
      super(nnModel, self).__init__()
      self.encoder_conv = nn.Sequential(
          nn.Conv2d(1, 8, 3, 2, 1),
          nn.BatchNorm2d(8),
          nn.ELU(inplace=True),

          nn.Conv2d(8, 32, 3, 1, 1),
          nn.BatchNorm2d(32),
          nn.ELU(inplace=True),

          nn.Conv2d(32, 64, 3, 1, 1),
          nn.BatchNorm2d(64),
          nn.ELU(inplace=True),

          nn.Conv2d(64, 128, 3, 1, 1),
          nn.BatchNorm2d(128),
          nn.ELU(inplace=True),
        )

      self.mean_layer = nn.Linear(128 * (image_size // 2) ** 2, 16)

      self.logvar_layer = nn.Linear(128 * (image_size // 2) ** 2, 16)

      self.decoder_conv = nn.Sequential(

          nn.ConvTranspose2d(16, 128, 3, 1, 1),
          nn.BatchNorm2d(128),
          nn.ELU(inplace=True),

          nn.ConvTranspose2d(128, 64, 3, 1, 1),
          nn.BatchNorm2d(64),
          nn.ELU(inplace=True),

          nn.ConvTranspose2d(64, 32, 3, 1, 1),
          nn.BatchNorm2d(32),
          nn.ELU(inplace=True),

          nn.ConvTranspose2d(32, 8, 3, 1, 1),
          nn.BatchNorm2d(8),
          nn.ELU(inplace=True),

          nn.Upsample(scale_factor=2, mode='nearest'),
          nn.ConvTranspose2d(8, 1, 3, 1, 1),
          nn.BatchNorm2d(1),
          nn.Sigmoid()
        )

    def forward(self, x):
      x = self.encoder_conv(x)
      x = x.view(x.size(0), -1)  # Flatten before linear layers
      mean, logvar = self.mean_layer(x), self.logvar_layer(x)
      epsilon = torch.randn_like(torch.exp(0.5 * logvar)).to(device)
      x = mean + logvar*epsilon
      x = self.decoder_conv(x.view(x.size(0), 16, (image_size // 2), (image_size // 2)))  # Reshape before decoding
      return x, mean, logvar



In [ ]:

with torch.no_grad():
    for data in test_loader:
        inputs, _ = data
        inputs = inputs.to(device)

        outputs = autoencoder(inputs)


        # Display original and reconstructed images
        n = 10  # Number of digits to display
        plt.figure(figsize=(20, 4))
        for i in range(n):
            # Original Images
            ax = plt.subplot(2, n, i + 1)
            plt.imshow(inputs[i].cpu().view(28, 28).numpy(), cmap='gray')
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            # Reconstructed Images
            ax = plt.subplot(2, n, i + 1 + n)
            plt.imshow(outputs[i].cpu().view(28, 28).numpy(), cmap='gray')
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

        plt.show()
        break
